# Qiskit VQE - SU(2) Matrix Model [Lambda = 4]

In [1]:
import numpy as np
import pylab
import matplotlib.pyplot as plt
from qiskit.circuit.library import TwoLocal, EfficientSU2

import sys
sys.path.append('../utility')
from vqe_run import *
from qc_ansatze import *

# Convert the Hamiltonian to strings of Pauli ops

In [3]:
# Load the H file already created using qiskit 0.28 from MBP 2015
file = open("../utility/pauliH_L4_g0.2.txt", "r")
content = file.read()
#print(content)

In [4]:
data = content.split(',\n')
data[0] = data[0].split('\n')[1]
data[-1] = data[-1].split('\n')[0]

data_coeff = []
data_str = []
for i in range(len(data)):
    data_coeff.append(float(data[i].split('*')[0]))
    data_str.append(data[i].split('*')[1].split(' ')[1])
    
Hpauli = list(zip(data_str,data_coeff))
#Hpauli

In [5]:
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import NumPyEigensolver
H4q = SparsePauliOp.from_list(Hpauli)

solver = NumPyEigensolver(k=4)
exact_solution = solver.compute_eigenvalues(H4q)
print("Exact Result of qubit hamiltonian:", np.real(exact_solution.eigenvalues))
E_exact = np.round(np.real(exact_solution.eigenvalues)[0],5)
E_exact

Exact Result of qubit hamiltonian: [3.13406307 4.21516634 4.21516634 4.21516634]


3.13406

# Circuit ansatze

In [6]:
#CIRCUIT ANSATZ
from qiskit.circuit.library import TwoLocal, EfficientSU2

In [6]:
reps=1
ansatz_0a = TL_ansatz(H4q.num_qubits, 'ry', 'crx', "circular", reps)
ansatz_0b = TL_ansatz(H4q.num_qubits, 'rz', 'crx', 'circular', reps)
ansatz_0c = TL_ansatz(H4q.num_qubits, ['ry','rz'], 'crx', 'circular', reps)
ansatz_0d = TL_ansatz(H4q.num_qubits,['ry','y'], 'crx', 'circular', reps)

ansatz_1a =  TL_ansatz(H4q.num_qubits, 'ry', 'crx', "full", reps)
ansatz_1b =  TL_ansatz(H4q.num_qubits, 'rz', 'crx', "full", reps)
ansatz_1c =  TL_ansatz(H4q.num_qubits, ['ry','rz'], 'crx', "full", reps)
ansatz_1d =  TL_ansatz(H4q.num_qubits, ['ry','y'], 'crx', "full", reps)

Circuit ansatz with 36 parameters
Circuit ansatz with 36 parameters
Circuit ansatz with 60 parameters
Circuit ansatz with 36 parameters
Circuit ansatz with 90 parameters
Circuit ansatz with 90 parameters
Circuit ansatz with 114 parameters
Circuit ansatz with 90 parameters


In [32]:
reps=1
ansatz_0a_su2 = ef_ansatz(H4q.num_qubits, 'ry',  "circular", reps)
ansatz_0b_su2 = ef_ansatz(H4q.num_qubits, 'rz',  'circular', reps)
ansatz_0c_su2 = ef_ansatz(H4q.num_qubits, ['ry','rz'], 'circular', reps)
ansatz_0d_su2 = ef_ansatz(H4q.num_qubits,['ry','y'], 'circular', reps)

ansatz_1a_su2 =  ef_ansatz(H4q.num_qubits, 'ry',  "full", reps)
ansatz_1b_su2 =  ef_ansatz(H4q.num_qubits, 'rz',  "full", reps)
ansatz_1c_su2 =  ef_ansatz(H4q.num_qubits, ['ry','rz'], "full", reps)
ansatz_1d_su2 =  ef_ansatz(H4q.num_qubits, ['ry','y'],  "full", reps)

Circuit ansatz with 24 parameters
Circuit ansatz with 24 parameters
Circuit ansatz with 48 parameters
Circuit ansatz with 24 parameters
Circuit ansatz with 24 parameters
Circuit ansatz with 24 parameters
Circuit ansatz with 48 parameters
Circuit ansatz with 24 parameters


# Qiskit QVE: Noiseless simulator

https://qiskit.github.io/qiskit-aer/stubs/qiskit_aer.primitives.Estimator.html

https://qiskit-community.github.io/qiskit-algorithms/tutorials/03_vqe_simulation_with_noise.html

In [7]:
#VQE ALGORITHMS
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SPSA
from qiskit_algorithms.utils import algorithm_globals 
from qiskit_aer.primitives import Estimator as AerEstimator

import warnings
warnings.filterwarnings("ignore")
seed = 170
iterations = 250
algorithm_globals.random_seed = seed

#estimator
noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": 1024},
    transpile_options={"seed_transpiler": seed},
)
#storing values
counts = []
values = []
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)
    
def run_qve_w_specified_optimizer(optimizer, ansatz):
    opt = optimizer(maxiter = iterations)
    vqe = VQE(noiseless_estimator, ansatz, optimizer=opt, callback=store_intermediate_result)
    result = vqe.compute_minimum_eigenvalue(operator=H4q).eigenvalue.real
    print(f"VQE result: {result:.5f}")
    return result

# COBYLA: EffSU2

In [33]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0a_su2)
counts_0a_c = counts
values_0a_c = values

VQE result: 3.29386


In [34]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0b_su2)
counts_0b_c = counts
values_0b_c = values

VQE result: 3.17910


In [35]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0c_su2)
counts_0c_c = counts
values_0c_c = values

VQE result: 7.03011


In [36]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0d_su2)
counts_0d_c = counts
values_0d_c = values

VQE result: 4.90114


In [37]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1a_su2)
counts_1a_c = counts
values_1a_c = values

VQE result: 3.33384


In [38]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1b_su2)
counts_1b_c = counts
values_1b_c = values

VQE result: 3.17910


In [39]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1c_su2)
counts_1c_c = counts
values_1c_c = values

VQE result: 9.76802


In [40]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1d_su2)
counts_1d_c = counts
values_1d_c = values

VQE result: 3.37175


In [42]:
df1 = pd.DataFrame({'values_0a_su2': values_0a_c})
df2 = pd.DataFrame({'values_0b_su2': values_0b_c})
df3 = pd.DataFrame({'values_0c_su2': values_0c_c})
df4 = pd.DataFrame({'values_0d_su2': values_0d_c})
df5 = pd.DataFrame({'values_1a_su2': values_1a_c})
df6 = pd.DataFrame({'values_1b_su2': values_1b_c})
df7 = pd.DataFrame({'values_1c_su2': values_1c_c})
df8 = pd.DataFrame({'values_1d_su2': values_1d_c})
qve_l2_tl_cobyla = pd.concat([df1, df2, df3, df4, df5, df6, df7,df8], axis = 1)
qve_l2_tl_cobyla.to_csv('qve_l4_effsu2_cobyla.csv')
qve_l2_tl_cobyla

,values_0a_su2,values_0b_su2,values_0c_su2,values_0d_su2,values_1a_su2,values_1b_su2,values_1c_su2,values_1d_su2
0,12.677996,3.1791,13.436468,12.466898,13.642510,3.1791,14.317269,14.956715
1,12.341952,3.1791,13.870390,12.346303,13.559569,3.1791,14.495946,15.018552
2,12.302492,3.1791,13.063490,12.475614,14.041793,3.1791,14.189223,13.405057
3,11.977049,3.1791,12.851695,12.722498,12.854220,3.1791,13.959487,13.454371
4,11.529180,3.1791,12.818057,12.453621,12.861947,3.1791,13.967840,13.812463
...,...,...,...,...,...,...,...,...
245,3.394844,NaN,7.260065,4.944397,3.357582,NaN,9.882192,3.435482
246,3.376981,NaN,7.157554,4.901141,3.346683,NaN,9.889194,3.399122
247,3.338841,NaN,7.263999,4.942851,3.358997,NaN,9.905328,3.404189
248,3.324259,NaN,7.278621,4.914533,3.346109,NaN,9.949311,3.402977


# COBYLA: TwoLocal

In [20]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0a)
counts_0a_c = counts
values_0a_c = values

VQE result: 3.13909


In [21]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0b)
counts_0b_c = counts
values_0b_c = values

VQE result: 3.17910


In [22]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0c)
counts_0c_c = counts
values_0c_c = values

VQE result: 3.31393


In [23]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0d)
counts_0d_c = counts
values_0d_c = values

VQE result: 3.19125


In [24]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1a)
counts_1a_c = counts
values_1a_c = values

VQE result: 3.49465


In [25]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1b)
counts_1b_c = counts
values_1b_c = values

VQE result: 3.17910


In [26]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1c)
counts_1c_c = counts
values_1c_c = values

VQE result: 4.04545


In [27]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1d)
counts_1d_c = counts
values_1d_c = values

VQE result: 3.58869


In [31]:
df1 = pd.DataFrame({'values_0a': values_0a_c})
df2 = pd.DataFrame({'values_0b': values_0b_c})
df3 = pd.DataFrame({'values_0c': values_0c_c})
df4 = pd.DataFrame({'values_0d': values_0d_c})
df5 = pd.DataFrame({'values_1a': values_1a_c})
df6 = pd.DataFrame({'values_1b': values_1b_c})
df7 = pd.DataFrame({'values_1c': values_1c_c})
df8 = pd.DataFrame({'values_1d': values_1d_c})
qve_l2_tl_cobyla = pd.concat([df1, df2, df3, df4, df5, df6, df7,df8], axis = 1)
qve_l2_tl_cobyla.to_csv('qve_l4_twolocal_cobyla.csv')
qve_l2_tl_cobyla

,values_0a,values_0b,values_0c,values_0d,values_1a,values_1b,values_1c,values_1d
0,16.308781,3.1791,9.140411,12.506715,15.077974,3.1791,16.508758,11.393470
1,15.745217,3.1791,9.451632,12.573342,14.896206,3.1791,16.231014,11.058210
2,16.467246,3.1791,8.378520,12.789515,13.575477,3.1791,16.946716,9.979358
3,15.508816,3.1791,8.419603,12.824306,13.979056,3.1791,15.990727,10.149444
4,14.132827,3.1791,8.821925,11.401583,13.940594,3.1791,15.065912,10.898567
...,...,...,...,...,...,...,...,...
245,3.193238,NaN,3.357654,3.212102,3.526458,3.1791,4.146805,3.684506
246,3.191451,NaN,3.336099,3.239781,4.146720,3.1791,4.349680,3.660849
247,3.189241,NaN,3.359436,3.198256,3.526312,3.1791,4.094111,3.657693
248,3.209546,NaN,3.368866,3.191643,3.844033,3.1791,4.206858,3.592731


# SPSA: EffSU2

In [43]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0a_su2)
counts_0a = counts
values_0a = values

VQE result: 3.23204


In [44]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0b_su2)
counts_0b = counts
values_0b = values

VQE result: 3.17910


In [45]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0c_su2)
counts_0c = counts
values_0c = values

VQE result: 4.89771


In [46]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0d_su2)
counts_0d = counts
values_0d = values

VQE result: 3.14605


In [47]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1a_su2)
counts_1a = counts
values_1a = values

VQE result: 3.19555


In [48]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1b_su2)
counts_1b = counts
values_1b = values

VQE result: 3.17910


In [49]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1c_su2)
counts_1c = counts
values_1c = values

VQE result: 3.11959


In [50]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1d_su2)
counts_1d = counts
values_1d = values

VQE result: 4.24528


In [51]:
# save the data to a dataframe 
list_l2 = {'values_0a_su2': values_0a, 'values_0b_su2': values_0b, 
        'values_0c_su2': values_0c, 'values_0d_su2': values_0d,
        'values_1a_su2': values_1a, 'values_1b_su2': values_1b, 
        'values_1c_su2': values_1c,'values_1d_su2': values_1d}
qve_l2_efsu2_spsa= pd.DataFrame(list_l2)
qve_l2_efsu2_spsa.to_csv('qve_l4_effsu2_spsa.csv')
qve_l2_efsu2_spsa

,values_0a_su2,values_0b_su2,values_0c_su2,values_0d_su2,values_1a_su2,values_1b_su2,values_1c_su2,values_1d_su2
0,13.207931,3.106743,13.146482,12.779282,13.303064,3.106743,13.584196,10.886075
1,12.574506,3.078833,13.650564,13.621923,14.731432,3.078833,13.793808,12.279064
2,12.754407,3.182943,13.312107,12.643588,14.365070,3.182943,14.108694,12.821878
3,12.994656,3.211858,13.552702,13.355027,13.544014,3.211858,13.676857,10.333940
4,13.168725,3.125087,13.075687,13.387932,14.134281,3.125087,13.779233,11.270636
...,...,...,...,...,...,...,...,...
546,3.426873,3.179395,5.122381,3.314605,3.371243,3.179395,3.336386,4.530537
547,3.247899,3.173587,5.172545,3.315539,3.364333,3.173587,3.414267,4.392452
548,3.406040,3.179395,5.125765,3.360249,3.393384,3.179395,3.388078,4.467156
549,3.309430,3.173587,5.077455,3.282302,3.314264,3.173587,3.240675,4.379643


# SPSA: TwoLocal

In [8]:
#3.18339
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0a)
counts_0a = counts
values_0a = values

VQE result: 3.18339


In [9]:
#3.17910
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0b)
counts_0b = counts
values_0b = values

VQE result: 3.17910


In [10]:
#3.62318
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0c)
counts_0c = counts
values_0c = values

VQE result: 3.62318


In [11]:
#3.13679
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0d)
counts_0d = counts
values_0d = values

VQE result: 3.13679


In [12]:
#3.16617
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1a)
counts_1a = counts
values_1a = values

VQE result: 3.16617


In [13]:
#3.17910
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1b)
counts_1b = counts
values_1b = values

VQE result: 3.17910


In [14]:
#4.10208
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1c)
counts_1c = counts
values_1c = values

VQE result: 4.10208


In [15]:
#4.26060
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1d)
counts_1d = counts
values_1d = values

VQE result: 3.14366


In [16]:
# save the data to a dataframe 
#don't reevaluate on a new run with just the new ansatze added
import pandas as pd
list_l2 = {'values_0a': values_0a, 'values_0b': values_0b, 
        'values_0c': values_0c, 'values_0d': values_0d,
        'values_1a': values_1a, 'values_1b': values_1b, 
        'values_1c': values_1c,'values_1d': values_1d}
qve_l2_tl_spsa= pd.DataFrame(list_l2)
qve_l2_tl_spsa.to_csv('qve_l4_twolocal_spsa.csv')
qve_l2_tl_spsa

,values_0a,values_0b,values_0c,values_0d,values_1a,values_1b,values_1c,values_1d
0,12.692804,3.106743,11.520565,11.291845,13.148210,3.106743,14.978004,15.952792
1,14.786100,3.078833,12.554792,8.891964,13.565912,3.078833,14.638826,17.722649
2,14.892741,3.182943,12.661869,10.569361,12.596718,3.182943,14.292682,17.329318
3,12.881686,3.211858,11.338885,10.040035,13.557275,3.211858,15.031778,16.351447
4,14.809839,3.125087,12.531833,10.211150,12.167282,3.125087,14.934180,17.559067
...,...,...,...,...,...,...,...,...
546,3.269568,3.179395,3.725496,3.237611,3.293454,3.179395,4.116064,3.351472
547,3.286283,3.173587,3.801690,3.242824,3.280994,3.173587,4.240540,3.294939
548,3.276364,3.179395,3.747287,3.294357,3.185476,3.179395,4.143066,3.256069
549,3.239224,3.173587,3.711513,3.304204,3.196859,3.173587,4.335627,3.236816
